# camel 5b

## setup environment

should print specs of computer

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


## load documents

read documents

In [2]:
# LOADING
# knows how to read/write info from/to the OS
import os
import json
from llama_index import SimpleDirectoryReader

# read the huggingf ace API
with open("../CONFIG_LIST.json", "r") as file:
    config = json.load(file)

# hugging api-key
os.environ['HUGGING_FACE_API_KEY'] = config["huggingFaceToken"]
HUGGING_FACE_API_KEY = os.environ.get("HUGGING_FACE_API_KEY")

# load the docs into memory
documents = SimpleDirectoryReader('../documents').load_data() # reads whole drectory.

## tokenise documents

In [3]:

from huggingface_hub import hf_hub_download

# select a model
model_id = "Writer/camel-5b-hf"
# https://huggingface.co/Writer/camel-5b-hf/tree/main

# select files from model repository
filenames = [
        "added_tokens.json", "config.json", "generation_config.json", 
        "special_tokens_map.json", "pytorch_model-00001-of-00003.bin", "pytorch_model-00002-of-00003.bin",
        "pytorch_model-00003-of-00003.bin", "pytorch_model.bin.index.json",
        "tokenizer_config.json", "vocab.json","tokenizer.json" 
]

# download it
# hf_hub_download modify to save in a folder ./models, git ignore
for filename in filenames:
    downloaded_model_path = hf_hub_download(
        repo_id=model_id,
        filename=filename,
        token=HUGGING_FACE_API_KEY
    )
    print(downloaded_model_path)

/home/jameslopes/git/HR_Assistant_DEWC/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/jameslopes/.cache/huggingface/hub/models--Writer--camel-5b-hf/snapshots/d1438e22a33b9115af0e47ab3a0fe844cbf588a6/added_tokens.json
/home/jameslopes/.cache/huggingface/hub/models--Writer--camel-5b-hf/snapshots/d1438e22a33b9115af0e47ab3a0fe844cbf588a6/config.json
/home/jameslopes/.cache/huggingface/hub/models--Writer--camel-5b-hf/snapshots/d1438e22a33b9115af0e47ab3a0fe844cbf588a6/generation_config.json
/home/jameslopes/.cache/huggingface/hub/models--Writer--camel-5b-hf/snapshots/d1438e22a33b9115af0e47ab3a0fe844cbf588a6/special_tokens_map.json
/home/jameslopes/.cache/huggingface/hub/models--Writer--camel-5b-hf/snapshots/d1438e22a33b9115af0e47ab3a0fe844cbf588a6/pytorch_model-00001-of-00003.bin
/home/jameslopes/.cache/huggingface/hub/models--Writer--camel-5b-hf/snapshots/d1438e22a33b9115af0e47ab3a0fe844cbf588a6/pytorch_model-00002-of-00003.bin
/home/jameslopes/.cache/huggingface/hub/models--Writer--camel-5b-hf/snapshots/d1438e22a33b9115af0e47ab3a0fe844cbf588a6/pytorch_model-00003-of-0

## 

## give the LLM a personality

In [4]:
# setup prompts - specific to StableLM
from llama_index.prompts import PromptTemplate

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate(
    "As a Human Resources expert"
    "write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{query_str}\n\n### Response:"
)

# prompt engineering - gets better answers

## embedding 

In [5]:
import torch
from llama_index.llms import HuggingFaceLLM
from llama_index import GPTVectorStoreIndex, PromptHelper, ServiceContext, LLMPredictor
from llama_index.embeddings import InstructorEmbedding
# from langchain.embeddings.huggingface import HuggingFaceEmbeddings
# can cammer

# sets up the LLM model with context
# each model is different
# set temp to 0 = Repeatability 
# device_map = depends on model 
llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=model_id,
    model_name=model_id,
    device_map="auto",
    tokenizer_kwargs={"max_length": 2048},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)

# this grenades the vector indexes 
# max_chunk_overlap = 10% overlap to keep context of where everything came from
# chunk_size_limit = tokens
# TODO: save index to vector DB (Chroma)
# max_input_size and context_window same size
embed_model = InstructorEmbedding(model_name="hkunlp/instructor-base")
max_input_size = 2048
num_output = 256
max_chunk_overlap = 0.1
chunk_size_limit = 600
prompt_helper = PromptHelper(max_input_size, num_output,max_chunk_overlap,chunk_size_limit=chunk_size_limit)
service_context = ServiceContext.from_defaults(llm=llm, prompt_helper=prompt_helper,embed_model=embed_model)

# index content in the folder documents
# saves the document to memory
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)
 
# Save your index to a directory called storage
# index.storage_context.persist(persist_dir="../storage")

INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 3/3 [00:12<00:00,  4.19s/it]


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: hkunlp/instructor-base
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
Use pytorch device: cuda


## answer qustions

In [6]:
# prompt = "when was DEWC launched"

# prompt = "How many females work for DEWC"

# query_engine = index.as_query_engine()
# response = query_engine.query(prompt)

# print(response)

copied camel to separate jupiter-book, read all 100 questions and answer them 

In [ ]:
import pandas as pd
import warnings

# Suppress warnings from the transformer model
warnings.filterwarnings('ignore')

# Read the contents of the questionnaire.txt file
with open('../questionnaire.txt', 'r') as file:
    questions = file.readlines()

# Initialize the query engine
query_engine = index.as_query_engine()

# Create a list to store the results
results_model = []

# Iterate over the questions and get answers from the query engine
for prompt in questions:
    prompt_cleaned = prompt.strip()
    
    # Query the engine and get the response
    response = query_engine.query(prompt_cleaned)
    
    # Append the question and response to the results list
    # Assuming 'model_id' is defined elsewhere in your script
    results_model.append([model_id, prompt_cleaned, response])
    
    # Print the question and the response
    print(f"Question: {prompt_cleaned}")
    print(f"Answer: {response}\n")

# Convert the results list into a pandas DataFrame
df = pd.DataFrame(results_model, columns=['Model ID', 'Question', 'Answer'])

# Save the DataFrame to a CSV file
df.to_csv('../results_model.csv', index=False)

print("Results saved to CSV file.")


In [ ]:
results_model

In [9]:
# 1: panda to csv
# merge
# new branch zepher